In [5]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import load_model

from skimage.transform import resize
from keras.models import model_from_json

In [6]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    #rescale 1./255
    proc_img = img / 255.0 
    img = (img - img_mean)/img_std
    proc_img = resize(img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weight_path)

    # evaluate loaded model on test data
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prob = model.predict(img)    
    prediction = (prob.ravel() >= thresh)
    return prediction, prob 

In [7]:
ds = pydicom.dcmread('test1.dcm')
ds

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows       

In [8]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "./my_model.json"
weight_path = "./xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.15

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred, prob = predict_image(my_model,img_proc,thresh)
    print("prediction", pred, "with pro", prob)
    
    

Load file test1.dcm ...
prediction [False] with pro [[0.12912504]]
Load file test2.dcm ...
prediction [False] with pro [[0.05535207]]
Load file test3.dcm ...
prediction [ True] with pro [[0.9905938]]
Load file test4.dcm ...
prediction [False] with pro [[0.12912504]]
Load file test5.dcm ...
prediction [False] with pro [[0.12912504]]
Load file test6.dcm ...
prediction [False] with pro [[0.12912504]]
